In [1]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

In [12]:
wikidata = pd.read_json('movies/data/wikidata-movies2.json.gz', orient='records', lines=True)
actors = pd.read_json('movies/data/actors.json.gz', orient='records', lines=True)
directors = pd.read_json('movies/data/directors.json.gz', orient='records', lines=True)

In [16]:
wikidata = wikidata[pd.notnull(wikidata['nbox'])]
#df = df[pd.notnull(df['made_profit'])]

In [20]:
# explode wikidata director
wikidata_d_stack = wikidata['director'].apply(pd.Series).stack().reset_index(level=1, drop=True).to_frame()
wikidata_d_stack.columns = ['wikidata_id']

In [21]:
wikidata_d = wikidata_d_stack.reset_index().merge(directors, how="left", on="wikidata_id").set_index('index')

In [22]:
#df_genre = pd.merge(df_genre, df, left_index=True, right_index=True).reset_index()

,wikidata_id,director_label
index,,
46,Q2622675,Aleksei Yuryevich German
99,Q137244,Alexei Uchitel
110,Q23844,George Clooney
687,Q944763,Peyton Reed
687,Q522057,Edgar Wright
843,Q51892574,Hatem Khraiche
953,Q4222061,Dmitriy Kiselev
953,Q4077720,Aleksandr Baranov
953,Q2833792,Alexander Kott
